In [ ]:
import requests
from collections import defaultdict
from functools import cache
import pandas as pd
from utils.utils import SimpleThreadPoolExecutor
import numpy as np
from psql_utils.epsql import get_schema, get_table_name, sanitize_table_name
from psql_utils import epsql

@cache
def engine():
    return epsql.Engine()

In [ ]:
def census_api_get(base_url, payload):
    payload = payload.copy() # Don't modify the original
    payload['key'] = open("secrets/census_api_key.txt").read().strip()
    response = requests.get(base_url, params=payload)
    if response.status_code != 200:
        print(response.status_code, response.text)
        response.raise_for_status()
    return pd.DataFrame(response.json()[1:], columns=response.json()[0])

class CensusApiDataset():
    def __init__(self, dataset_name):
        self.dataset_name = dataset_name
        self.schema_name = "census"
        unsorted_vars = requests.get(f'https://api.census.gov/data/{dataset_name}/variables.json').json()['variables']
        self.variables = dict(sorted(unsorted_vars.items()))
        tables_build = defaultdict(dict)
        for var_name, var_info in self.variables.items():
            for table_name in set(var_info['group'].split(',')) - {'N/A'}:
                tables_build[table_name][var_name]=var_info

        self.tables = dict(sorted(tables_build.items()))
        print(f"{dataset_name}: found tables ({', '.join(self.tables.keys())})")
    
    def get_data(self, fields: list[str], geometry_level: str, in_: str = ""):
        if geometry_level == "blockgroup":
            geometry_level = "block group"
        assert(fields)
        data: pd.DataFrame|None = None
        shards: list[pd.DataFrame] = []
        # Observe the census api limit of 50 fields per request, and make sure to add GEO_ID
        # to each subrequest so we can join the results
        for subfields in [fields[i:i + 49] for i in range(0, len(fields), 49)]:
            if "GEO_ID" not in subfields:
                subfields.append("GEO_ID")                
            payload = {
                'get': ','.join(subfields),
                'for': f'{geometry_level}:*'
            }
            if in_:
                payload['in'] = in_
        
            subdata = census_api_get(f'https://api.census.gov/data/{self.dataset_name}', payload)

            if data is None:
                data = subdata
            else:
                # Join on all columns that are in both dataframes, as merge will not duplicate and add suffixes
                # to any columns in the "on" argument.
                # We assume that any duplicate columns (e.g. state, county) have the same data, and that GEO_ID
                # in particular is specific enough to guarantee like records are combined.
                join_columns = list(set(subdata.columns).intersection(set(data.columns)))
                data = data.merge(subdata, on=join_columns)

        assert(data is not None)

        for col in data.columns:
            var_info = self.variables.get(col)
            if var_info:
                dtypes = {
                    'int': np.int32,
                    'string': object,
                    'float': np.float32,
                    
                }
                new_dtype = dtypes[var_info['predicateType']]
                if data[col].dtype != new_dtype:
                    data[col] = data[col].astype(dtypes[var_info['predicateType']])
    
        if 'GEO_ID' not in fields:
            data.drop(columns=['GEO_ID'], inplace=True)

        if "block group" in data.columns:
            data.rename(columns={"block group": "blockgroup"}, inplace=True)

        if "GEO_ID" in data.columns:
            data['geoid'] = data['GEO_ID'].str[9:]
        return data
    
    @cache
    def get_states(self):
        states = self.get_data(["NAME", "GEO_ID"], "state")
        return dict(sorted(zip(states["geoid"], states.to_dict('records'))))

    @cache
    def get_counties(self):
        counties = self.get_data(["NAME", "GEO_ID"], "county")
        return dict(sorted(zip(counties["geoid"], counties.to_dict('records'))))

    def sql_table_name(self, table_name: str, geometry_level: str):
        tokens = self.dataset_name.split("/")
        assert(len(tokens) == 3)
        (year, dataset, subfile) = tokens
        return sanitize_table_name(f"{self.schema_name}.{dataset}{year}{subfile}_{table_name}_{geometry_level}")
    
    # Download all records for table_name
    def download_table(self, table_name: str, geometry_level: str):
        sql_table_name = self.sql_table_name(table_name, geometry_level)
        print(f"Downloading {sql_table_name}")
        engine().execute(f"CREATE SCHEMA IF NOT EXISTS {get_schema(sql_table_name)}")

        assert table_name in self.tables
        assert geometry_level in ['block', 'blockgroup', 'tract', 'county', 'place', 'state']
        fields = []
        for var_name, var_info in self.tables[table_name].items():
            fields.append(var_name)
            fields += var_info['attributes'].split(',')
        shards = []
        if geometry_level == 'tract':
            downloads = [{
                "sql":f"geoid between '{geoid}' and '{geoid}z'",
                "in":f"state:{geoid}"
            } for geoid in self.get_states().keys()]
        elif geometry_level in ['blockgroup', 'block']:
            downloads = [{
                "sql":f"geoid between '{geoid}' and '{geoid}z'",
                "in":f"state:{geoid[0:2]} county:{geoid[2:5]}"
            } for geoid in self.get_counties().keys()]
        else:
            downloads = [{}]
        n_done = 0
        for download in downloads:
            sql = f"SELECT count(*) from {sql_table_name}"
            if "sql" in download:
                sql += f" WHERE {download['sql']}"
            in_ = download.get("in", "")
            if engine().table_exists(sql_table_name) and (count := engine().execute_returning_value(sql)) > 0:
                n_done += 1
                #print(f"{sql_table_name} {in_} already loaded ({count} records), skipping")
                continue
            table = self.get_data(fields, geometry_level, in_)
            table.to_sql(get_table_name(sql_table_name), engine().engine, schema=get_schema(sql_table_name), if_exists='append', index=False)
            if not engine().table_has_primary_key(sql_table_name):
                engine().execute(f'ALTER TABLE {sql_table_name} ADD PRIMARY KEY (geoid)')
            print(f"{sql_table_name} {in_} loaded {len(table)} records of {len(table.columns)} fields")
        if n_done:
            print(f"{sql_table_name} {n_done} of {len(downloads)} downloads previously completed")
        print(f"{sql_table_name} is complete")

    def download_table_geometries(self, table_name: str):
        for geometry_level in ['state', 'county', 'tract', 'blockgroup', 'block']:
            self.download_table(table_name, geometry_level)

    def download_tables_geometries(self, nthreads: int = 10):
        pool = SimpleThreadPoolExecutor(nthreads)
        print(f"Downloading tables {', '.join(self.tables.keys())}")
        for table_name in self.tables.keys():
            pool.submit(self.download_table_geometries, table_name)
        pool.shutdown()

#engine().execute("drop table census.dec2020pl_p2_tract")

for dataset in [
    "2020/dec/pl",
    "2010/dec/sf1", "2010/dec/sf2",
    "2000/dec/sf1", "2000/dec/sf2", "2000/dec/sf3"]:
    ds = CensusApiDataset(dataset)
    ds.download_tables_geometries()



